# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [8]:
import pandas as pd
from xml.etree import ElementTree as ET

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [2]:
document_tree = ET.parse( './data/mondial_database_less.xml' )

In [3]:
# print names of all countries
for child in document_tree.getroot():
    print (child.find('name').text)

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [4]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    #print ('* ' + element.find('name').text + ':',)
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print ('* ' + element.find('name').text + ':' + capitals_string[:-2])

* Albania:Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece:Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia:Skopje, Kumanovo
* Serbia:Beograd, Novi Sad, Niš
* Montenegro:Podgorica
* Kosovo:Prishtine
* Andorra:Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

In [4]:
#load dataset
document = ET.parse( './data/mondial_database.xml' )

# Question 1: 10 countries with the lowest infant mortality rates

In [10]:
country_names = []
infant_mortality = []
for element in document.iterfind('country'):
    #Ignore observations without infant_mortality data
    if element.find('infant_mortality') == None:
        continue
    #Append contry name to contry_names
    country_names.append(element.find('name').text)
    #Append infant_mortality to list infant_mortality
    infant_mortality.append(element.find('infant_mortality').text)

#Create a dictionary with country_names as keys and infant_mortality as values
data_q1 = dict(zip(country_names, infant_mortality))

In [12]:
#Convert values of dictionary(data_q1) to float and sort the result
data_q1_ans = sorted(data_q1.items(), key=lambda x:float(x[1]))
#Convert dict to dataframe
df = pd.DataFrame(data_q1_ans)
#Change column names
df.columns = (['Country', 'Infant Mortality'])
df.head(10)

,Country,Infant Mortality
0,Monaco,1.81
1,Japan,2.13
2,Bermuda,2.48
3,Norway,2.48
4,Singapore,2.53
5,Sweden,2.6
6,Czech Republic,2.63
7,Hong Kong,2.73
8,Macao,3.13
9,Iceland,3.15


# Question 2: 10 cities with the largest population

In [13]:
result = []
for element in document.iterfind('country'):
    country = element.find('name').text
    for subelement in element.getiterator('city'):
        #Concatenate city name with country name to show them together
        city = subelement.find('name').text + ', ' + country
        #This inner for loop is aimed to use the latest population data and its year as our data
        for population in subelement.getiterator('population'):
            city_pop = '{:15,.0f}'.format(int(population.text))
            year = population.attrib['year']
        #Create a dictionary contains the data we created
        result.append({'cities': city, 'population': city_pop, 'year': year})

#Change list to dataframe
result = pd.DataFrame(result)

In [14]:
#Sort by 'population' from highest to lowest and make it look better
result = result.sort('population', ascending = False).reset_index().drop(['index'], axis = 1)
result.head(10)

/home/yangzhibo5947/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


,cities,population,year
0,"Shanghai, China","22,315,474",2010
1,"Istanbul, Turkey","13,710,512",2012
2,"Mumbai, India","12,442,373",2011
3,"Moskva, Russia","11,979,529",2013
4,"Beijing, China","11,716,620",2010
5,"São Paulo, Brazil","11,152,344",2010
6,"Tianjin, China","11,090,314",2010
7,"Guangzhou, China","11,071,424",2010
8,"Delhi, India","11,034,555",2011
9,"Shenzhen, China","10,358,381",2010


# Question 3: 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)

In [19]:
result_q3 = []
for country in document.findall('country'):
    for node in list(country):
        if node.tag == 'name':
            country = node.text
        #This make sure that we are using the latest data in the data set
        elif node.tag == 'population':
            pop = int(node.text)
            yearr = int(node.attrib['year'])
        elif node.tag == 'ethnicgroup':
            ethnic = node.text
            percentage = float(node.attrib['percentage'])
            #Multiply population with percentage of ethnicgroups
            #This will result in the population of certain ethnicgroups
            eth_pop = int(pop * percentage / 100.)
            
            #Store the result dict to a list result_q3
            result_q3.append({'countries':country, 'country_pop':pop, 'year':year,
                        'ethnicity':ethnic, 'country_percentage':percentage, 'population':eth_pop})

#Change list to dataframe
result_q3 = pd.DataFrame(result_q3)

/home/yangzhibo5947/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


In [22]:
result_q3 = result_q3.sort('population', ascending = False)
result_q3.head(10)

/home/yangzhibo5947/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,countries,country_percentage,country_pop,ethnicity,population,year
176,China,91.50,1360720000,Han Chinese,1245058800,2002
221,India,72.00,1210854977,Indo-Aryan,871815583,2002
220,India,25.00,1210854977,Dravidian,302713744,2002
345,United States,79.96,318857056,European,254958101,2002
520,Nigeria,99.00,164294516,African,162651570,2002
212,Bangladesh,98.00,149772364,Bengali,146776916,2002
299,Japan,99.40,127298000,Japanese,126534212,2002
93,Russia,79.80,143666931,Russian,114646210,2002
278,Indonesia,45.00,252124458,Javanese,113456006,2002
461,Brazil,53.70,202768562,European,108886717,2002


# Question 4. name and country of 
# a) longest river ,
# b) largest lake and,
# c) airport at highest elevation

# 1. Longest River

In [24]:
#Check the nodes' tag in our data set
tag_list = []
for element in document.findall('river'):
    for node in list(element):
        tag_list.append(node.tag)
{node : 1 for node in tag_list}.keys()

dict_keys(['source', 'area', 'through', 'name', 'located', 'length', 'to', 'estuary'])

In [32]:
river_data = []
for element in document.findall('river'):
    country = element.attrib['country'].split(' ')[0]
    for node in list(element):
        if node.tag == 'name':
            river = node.text
        elif node.tag == 'length':
            length = float(node.text)
            river_data.append({'river': river, 'length': length, 'country code': country})

river_country = pd.DataFrame(river_data)

In [33]:
country_code = []
for element in document.findall('country'):
    #car_code = element.attrib['car_code']
    for node in list(element):
        if node.tag == 'name':
            car_code = element.attrib['car_code']
            country = node.text
            country_code.append({'car code': car_code, 'country': country})
            
country_code = pd.DataFrame(country_code)

In [34]:
pd.merge(river_country, country_code, left_on = 'country code', right_on ='car code', how = 'left')\
  .sort('length', ascending = False).head(10)

/home/yangzhibo5947/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,country code,length,river,car code,country
174,CO,6448.0,Amazonas,CO,Colombia
137,CN,6380.0,Jangtse,CN,China
136,CN,4845.0,Hwangho,CN,China
123,R,4400.0,Lena,R,Russia
201,RCB,4374.0,Zaire,RCB,Congo
138,CN,4350.0,Mekong,CN,China
115,R,4248.0,Irtysch,R,Russia
186,RMM,4184.0,Niger,RMM,Mali
160,USA,4130.0,Missouri,USA,United States
119,R,4092.0,Jenissej,R,Russia


# 2. Largest Lake

In [28]:
lake_data = []
for element in document.findall('lake'):
    country = element.attrib['country'].split(' ')[0]
    for node in list(element):
        if node.tag == 'name':
            lake = node.text
        elif node.tag == 'area':
            area = float(node.text)
            
            lake_data.append({'country': country, 'lake': lake, 'area': area})
    
lake_df = pd.DataFrame(lake_data)

In [29]:
pd.merge(lake_df, co_co, left_on = 'country', right_on ='car code', how = 'left')\
  .sort('area', ascending = False).head(10)

/home/yangzhibo5947/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,area,country_x,lake,car code,country_y
54,386400.0,R,Caspian Sea,R,Russia
107,82103.0,CDN,Lake Superior,CDN,Canada
79,68870.0,EAT,Lake Victoria,EAT,Tanzania
104,59600.0,CDN,Lake Huron,CDN,Canada
106,57800.0,USA,Lake Michigan,USA,United States
47,41650.0,IL,Dead Sea,IL,Israel
81,32893.0,ZRE,Lake Tanganjika,ZRE,Zaire
96,31792.0,CDN,Great Bear Lake,CDN,Canada
43,31492.0,R,Ozero Baikal,R,Russia
87,29600.0,MW,Lake Malawi,MW,Malawi


# 3. Airport At Highest Evaluation

In [30]:
airport_data = []
for element in document.findall('airport'):
    country = element.attrib['country'].split(' ')[0]
    for node in list(element):
        if node.tag == 'name':
            airport = node.text
        elif node.tag == 'elevation':
            elevation = node.text
            
            airport_data.append({'country': country, 'airport': airport, 'elevation': elevation})
            
airport_df = pd.DataFrame(airport_data)

In [31]:
pd.merge(airport_df, co_co, left_on = 'country', right_on ='car code', how = 'left')\
  .sort('elevation', ascending = False).head(10)

/home/yangzhibo5947/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':


,airport,country_x,elevation,car code,country_y
536,Mashhad,IR,995,IR,Iran
915,Yakutsk,R,99,R,Russia
1009,A Coruna,E,99,E,Spain
387,Guipavas,F,99,F,France
498,Allahabad,IND,98,IND,India
361,Kuopio,SF,98,SF,Finland
1280,Urgench Airport,UZB,98,UZB,Uzbekistan
0,Herat,AFG,977,AFG,Afghanistan
285,Alfonso Bonilla Aragon Intl,CO,964,CO,Colombia
864,Henri Coanda,RO,96,RO,Romania
